In [ ]:
https://github.com/townblack/pytorch-cifar10-resnet18

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
from resnet import ResNet18
import time
from tensorboardX import SummaryWriter

'''
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
'''
# Define whether to use GPU
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Argument settings to allow command-line input manually, making the style similar to Linux CLI
parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
parser.add_argument('--outf', default='./model/', help='folder to output images and model checkpoints') # Path to save output results
parser.add_argument('--net', default='./model/Resnet18.pth', help="path to net (to continue training)")  # Path to the model for resuming training
args = parser.parse_args()

# Hyperparameter settings
EPOCH = 240
BATCH_SIZE = 128
LR = 0.001        #learning rate
Milestones=[135,185]
Debug=False

# Prepare and preprocess the dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # First pad the image with zeros around, then randomly crop it to 32x32
    transforms.RandomHorizontalFlip(), # Random horizontal flip with 50% chance
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), # Mean and std used for normalizing each R, G, B channel
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data/cifar-10-python', train=True, download=False, transform=transform_train) # Training dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)   # Generate batches for training, shuffle data when forming batches

testset = torchvision.datasets.CIFAR10(root='./data/cifar-10-python', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
# Cifar-10's label
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model definition - ResNet
net = ResNet18().to(device)

# Define loss function and optimization method
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss function, commonly used for multi-class classification
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) # Optimization method is mini-batch momentum-SGD with L2 regularization (weight decay)


scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=Milestones,gamma = 0.1)

writer=SummaryWriter("./logs")

# train
if __name__ == "__main__":
    best_acc = 85  #2 initialize best test accuracy
    print("Start Training, Resnet-18!")  # Define number of epochs to iterate over the dataset
    with open("acc.txt", "w") as f:
        with open("log.txt", "w")as f2:
            for epoch in range(EPOCH):
                train_loss=0.0
                train_accu=0.0
                val_loss=0.0
                val_accu=0.0

                scheduler.step()
                #print(type(optimizer.param_groups[0]))
                #print(optimizer.param_groups[0]["lr"])
                #print('\nEpoch: %d' % (epoch + 1))
                net.train()
                sum_loss = 0.0
                correct = 0.0
                total = 0
                begin=time.time()
                for i, data in enumerate(trainloader, 0):
                    # prepare data
                    length = len(trainloader)
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    # forward + backward
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    # Print loss and accuracy after training each batch
                    sum_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += predicted.eq(labels.data).cpu().sum()
                    if Debug:
                        print("[Epoch:{}/{}, Batch:{}/{}] Loss: {:.3f} | Acc: {:.3f}%".format(epoch+1,EPOCH,i+1,int(trainset.__len__()/BATCH_SIZE),sum_loss/(i+1),100.*correct/total))

                    f2.write("[Epoch:{}/{}, Batch:{}/{}] Loss: {:.3f} | Acc: {:.3f}%".format(epoch+1,EPOCH,i+1,int(trainset.__len__()/BATCH_SIZE),sum_loss/(i+1),100.*correct/total))
                    f2.write('\n')
                    f2.flush()

                train_loss=sum_loss/int(trainset.__len__()/BATCH_SIZE)
                train_accu=100.*correct/total

               # Evaluate accuracy after each epoch
                with torch.no_grad():
                    sum_loss = 0.0
                    correct = 0.0
                    total = 0
                    for data in testloader:
                        net.eval()
                        images, labels = data
                        images, labels = images.to(device), labels.to(device)
                        outputs = net(images)
                        loss = criterion(outputs, labels)
                        sum_loss += loss.item()
                       # Get the class with the highest score (index of outputs.data)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum()

                val_loss=sum_loss/int(testset.__len__()/BATCH_SIZE)
                val_accu = 100.*correct/total
                end=time.time()
                print("[Epoch:{}/{}] Train Loss: {:.3f} | Train Acc: {:.3f}% Test Loss: {:.3f} | Test Acc: {:.3f}% Cost time:{:.2f}min".format(epoch+1,EPOCH,train_loss,train_accu,val_loss,val_accu,(end-begin)/60.0))

                writer.add_scalar("Loss/train",train_loss,epoch)
                writer.add_scalar("Loss/val",val_loss,epoch)
                writer.add_scalar("Accu/train",train_accu,epoch)
                writer.add_scalar("Accu/val",val_accu,epoch)
                writer.add_scalar("Learning rate",optimizer.param_groups[0]["lr"],epoch)

                # Write each evaluation result to acc.txt in real time
                #print('Saving model......')
                torch.save(net.state_dict(), '%s/net_%03d.pth' % (args.outf,epoch + 1))
                f.write("EPOCH=%03d,Accuracy= %.3f%%" % (epoch+1,val_accu))
                f.write('\n')
                f.flush()
                # Record the best test accuracy and save to best_acc.txt
                if val_accu > best_acc:
                    f3 = open("best_acc.txt", "w")
                    f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch+1,val_accu))
                    f3.close()
                    best_acc = val_accu

            print("Training Finished, TotalEPOCH=%d" % EPOCH)

ModuleNotFoundError: No module named 'resnet'